In [2]:
import os
import re

import pystac
import pystac.item
import pystac.link
from pystac.provider import ProviderRole
from pystac.extensions.projection import ProjectionExtension

from datetime import datetime, UTC

In [3]:
COLLITAS_TITLE_DICT = {
    'clc_map': 'Corine Land Cover Map',
    'clc_map_statistics': 'Corine Land Cover Map Statistics',
    'clc_map_pyramid': 'Pyramid',
    'clc_map_encoding': 'Encoding',
    'clc_map_database': 'Database',
    'clc_map_database_metadata': 'Database Metadata',
    'clc_map_tif_metadata': 'TIFF Metadata',
    'clc_map_metadata': 'Corine Land Cover Map Metadata',
}

COLLITAS_MEDIA_TYPE_DICT = {
    'clc_map': pystac.MediaType.COG,
    'clc_map_statistics': pystac.MediaType.XML,
    'clc_map_pyramid': 'image/tiff; application=geotiff; profile=pyramid',
    'clc_map_encoding': pystac.MediaType.TEXT,
    'clc_map_database': 'application/dbf',
    'clc_map_database_metadata': pystac.MediaType.TEXT,
    'clc_map_tif_metadata': 'image/tiff; application=geotiff; profile=layer',
    'clc_map_metadata': pystac.MediaType.XML,
}

COLLITAS_ROLES_DICT = {
    'clc_map': ['data'],
    'clc_map_statistics': ['metadata'],
    'clc_map_pyramid': ['metadata'],
    'clc_map_encoding': ['metadata'],
    'clc_map_database': ['metadata'],
    'clc_map_database_metadata': ['metadata'],
    'clc_map_tif_metadata': ['metadata'],
    'clc_map_metadata': ['metadata'],
}

In [4]:
WORKING_DIR = os.getcwd()
STAC_DIR = 'stac_tests'
COLLECTION_ID = 'corine-land-cover-raster'
COLLECTION_TITLE = 'CORINE Land Cover Raster'
COLLECTION_DESCRIPTION = ("The European Commission launched the CORINE (Coordination of Information on the Environment) "
                          "program in an effort to develop a standardized methodology for producing continent-scale land "
                          "cover, biotope, and air quality maps. The CORINE Land Cover (CLC) product offers a pan-European "
                          "land cover and land use inventory with 44 thematic classes, ranging from broad forested areas "
                          "to individual vineyards.")
COLLECTION_KEYWORDS =  ["clms", "corine", "derived data", "land cover", "machine learning", "open data"]
COLLECTION_LICENSE = 'proprietary'


# CLMS_CATALOG_LINK = pystac.link.Link(
#     rel=pystac.RelType.ROOT, target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, "stacs/clms_catalog.json"))
# )
# COLLECTION_LINK = pystac.link.Link(
#     rel=pystac.RelType.COLLECTION,
#     target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, f"stacs/{COLLECTION_ID}/{COLLECTION_ID}.json")),
# )
# ITEM_PARENT_LINK = pystac.link.Link(
#     rel=pystac.RelType.PARENT,
#     target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, f"stacs/{COLLECTION_ID}/{COLLECTION_ID}.json")),
# )

In [5]:

sp_extent = pystac.SpatialExtent([None, None, None, None])
tmp_extent = pystac.TemporalExtent([datetime(1990, 1, 1, microsecond=0, tzinfo=UTC), None])
extent = pystac.Extent(sp_extent,  tmp_extent)

collection = pystac.Collection(id=COLLECTION_ID,
                               description=COLLECTION_DESCRIPTION,
                               title=COLLECTION_TITLE,
                               extent=extent,
                               keywords=COLLECTION_KEYWORDS,
                               license=COLLECTION_LICENSE
                               )

In [6]:
collection

<Collection id=corine-land-cover-raster>

In [7]:
from pystac.extensions.item_assets import ItemAssetsExtension, AssetDefinition

item_assets = ItemAssetsExtension.ext(collection, add_if_missing=True)
item_assets.item_assets = {
    key: AssetDefinition({"title": COLLITAS_TITLE_DICT[key].format(label='').strip(),
                           "media_type": COLLITAS_MEDIA_TYPE_DICT[key], 
                           "roles": COLLITAS_ROLES_DICT[key]})
    for key in COLLITAS_TITLE_DICT
}

In [8]:
collection

<Collection id=corine-land-cover-raster>